In [1]:
# Analysis Community Modules
import pandas as pd
import gseapy as gp 
from pathlib import Path
import plotly.graph_objects as go
from scipy.stats import fisher_exact, hypergeom
import numpy as np
from scipy.stats import false_discovery_control


In [13]:
#load in file matching node IDs to gene names
path_vertices = Path("/Users/paulinestaiber/Documents/Network/Multiplex-Network-Community-detection/results/multiplex_clustering_vertices.tsv")
vertices_df = pd.read_csv(path_vertices, sep="\t", header=0, names=["node:id", "node_name"])
vertices_df

,node:id,node_name
0,1,MED25
1,2,ENPP1
2,3,EIF2S2
3,4,LIPH
4,5,UHMK1
...,...,...
15256,15257,PAPOLG
15257,15258,VANGL1
15258,15259,PROSER1
15259,15260,RAB3IL1


In [14]:
#load community detection results and subset dataframe to only node IDs and module assignments
path = Path("/Users/paulinestaiber/Documents/Network/Multiplex-Network-Community-detection/cluster_output_infomap/multiplex_clustering.clu")
agg = pd.read_csv(path, comment="#", header=None, sep=" ", names=[ "node_id","module_merged","flow" ])
agg_small = agg[["node_id", "module_merged"]]
agg_small
#number of modules 
num_modules = agg_small["module_merged"].nunique()
print(f"Number of modules: {num_modules}")



Number of modules: 118


In [15]:
# remove duplicates
agg_small = agg_small.drop_duplicates()
len(agg_small)
num_nodes = agg_small["node_id"].nunique()
num_nodes

15261

In [16]:
# merge agg_small with vertices_df to get gene names
merge_df = agg_small.merge(vertices_df, left_on="node_id", right_on="node:id", how="left")
merge_df = merge_df.drop(columns=["node:id"])
merge_df




,node_id,module_merged,node_name
0,63,1,BCKDK
1,169,1,DNAJC1
2,199,1,TMEM246
3,222,1,PLP2
4,449,1,TOMM7
...,...,...,...
15379,13161,115,HARBI1
15380,2485,116,HMX3
15381,10858,116,HMX2
15382,12279,117,CLC


In [17]:
#safe the merged dataframe containing node IDs, gene names and module assignments
merge_df.to_csv("/Users/paulinestaiber/Documents/Network/Multiplex-Network-Community-detection/results/merged_dataframe.tsv", sep="\t", index=False)

In [18]:
#read txt file containing disease genes
with open("/Users/paulinestaiber/Documents/Network/Multiplex-Network-Community-detection/Data/raw/disease_genes.txt", "r") as f:
    gene_list = f.read().splitlines()

In [ ]:
# 1. Count disease genes per module
disease_in_modules = merge_df[merge_df["node_name"].isin(gene_list)].groupby("module_merged")["node_name"].nunique().reset_index()
disease_in_modules.columns = ["module_merged", "disease_genes_count"]

# 2. Count all genes per module
total_in_modules = merge_df.groupby("module_merged")["node_name"].nunique().reset_index()
total_in_modules.columns = ["module_merged", "total_genes_count"]

# 3. Merge
module_stats = disease_in_modules.merge(total_in_modules, on="module_merged")

# 4. Total genes and total disease genes
total_genes = merge_df["node_name"].nunique()
total_disease_genes = len(gene_list)

# 5. Hypergeometric test for each module
p_values = []
for _, row in module_stats.iterrows():
    k = row["disease_genes_count"]  # Disease genes in module
    M = total_genes                  # All genes in network
    n = total_disease_genes         # All disease genes
    N = row["total_genes_count"]    # Total genes in module
    
    # P-value: Probability of seeing k or more disease genes in module
    p_val = hypergeom.sf(k-1, M, n, N)
    p_values.append(p_val)

module_stats["p_value"] = p_values
# Enrichment: (observed proportion of disease genes in module) / (expected proportion of disease genes)
module_stats["enrichment"] = (module_stats["disease_genes_count"] / module_stats["total_genes_count"]) / (total_disease_genes / total_genes)

module_stats = module_stats.sort_values("p_value")
module_stats



In [23]:
# Benjamini-Hochberg multiple testing correction
module_stats["p_value_fdr"] = false_discovery_control(module_stats["p_value"])

module_stats = module_stats.sort_values("p_value_fdr")
module_stats
module_stats.to_csv("/Users/paulinestaiber/Documents/Network/Multiplex-Network-Community-detection/results/dis_gene_enrichment_stats.tsv", sep="\t", index=True)


In [24]:
#filter for modules sig enriched with disease genes (FDR < 0.05)

modules_sig_enriched = module_stats[module_stats["p_value_fdr"] < 0.05]
modules_sig_enriched


,module_merged,disease_genes_count,total_genes_count,p_value,enrichment,p_value_fdr
6,11,13,246,3.893724e-15,23.04216,4.283096e-14


In [26]:
modules_sig_enriched.to_csv(
    "/Users/paulinestaiber/Documents/Network/Multiplex-Network-Community-detection/results/sig_enriched_modules_stats.csv",
    index=True
)